# LAB 5: AGENTIC AI - COMPLETE IMPLEMENTATION

**Course:** Advanced Prompt Engineering Training  
**Session:** Session 5 - Agentic AI (Day 3)  
**Duration:** 120 minutes (2 hours)  
**Type:** Comprehensive Agent Development Workshop

## LAB OVERVIEW

This comprehensive lab teaches you to build **production-grade agentic AI systems** using LangChain. You'll progress through six interconnected modules:

1. **Basic ReAct Agent** - Understand the thought-action-observation loop
2. **Custom Tools** - Build domain-specific tools for agents
3. **Multi-Tool Agent** - Combine tools for complex problem-solving
4. **Conversational Memory** - Add stateful, context-aware behavior
5. **Prompt Optimization** - Improve reliability and reduce errors
6. **Advanced Patterns** - Plan-and-execute, self-correction, error handling

**Scenario:** You're building an AI financial advisor assistant for a bank that can answer customer questions, look up account information, calculate financial metrics, and provide personalized recommendations using autonomous reasoning and tool use.

## LEARNING OBJECTIVES

By the end of this lab, you will be able to:

✓ **Build ReAct agents** that reason and use tools autonomously  
✓ **Create custom tools** with proper descriptions and error handling  
✓ **Integrate multiple tools** for complex problem-solving  
✓ **Add conversation memory** for stateful interactions  
✓ **Optimize agent prompts** for reliability and accuracy  
✓ **Implement advanced patterns** like plan-and-execute  
✓ **Deploy production-ready agents** with proper guardrails

## SETUP INSTRUCTIONS

### Step 1: Import Libraries

In [ ]:
# Lab 5: Agentic AI Implementation
# Advanced Prompt Engineering Training - Session 5

import os
import json
import time
from datetime import datetime
from typing import List, Dict, Tuple, Optional, Any
from dataclasses import dataclass, field

# LangChain core
from langchain_classic.agents import create_react_agent, AgentExecutor, Tool
from langchain_classic.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_classic.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_classic.callbacks import StdOutCallbackHandler

from dotenv import load_dotenv

# Tools
from langchain_community.tools import DuckDuckGoSearchRun
import chromadb

load_dotenv(override=True)

print("✓ All libraries imported successfully")

### Step 2: Configure OpenAI and Models

In [ ]:
# Initialize OpenAI
from openai import OpenAI

api_key=os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable is not set. Please set it to your OpenAI API key.")

model_name = os.environ.get("MODEL_NAME", "gpt-4o")

if model_name not in ["gpt-4o", "gpt-4", "gpt-3.5-turbo"]:
    raise ValueError(f"Unsupported MODEL_NAME '{model_name}'. Supported models are: gpt-4o, gpt-4, gpt-3.5-turbo.")

fastmodel_name = os.environ.get("FAST_MODEL_NAME", "gpt-3.5-turbo")

if fastmodel_name not in ["gpt-4o", "gpt-4", "gpt-3.5-turbo"]:
    raise ValueError(f"Unsupported FAST_MODEL_NAME '{fastmodel_name}'. Supported models are: gpt-4o, gpt-4, gpt-3.5-turbo.")

client = OpenAI(api_key=api_key)

# LangChain models
llm_gpt4 = ChatOpenAI(model=model_name, temperature=0)
llm_gpt35 = ChatOpenAI(model=fastmodel_name, temperature=0)

print(f"✓ Models configured: {model_name}, {fastmodel_name}")

### Step 3: Create Mock Financial Database

In [ ]:
# Mock financial database for agent tools

MOCK_CUSTOMER_DATABASE = {
    "CUST001": {
        "name": "Sarah Johnson",
        "account_type": "Premium Checking",
        "balance": 45230.50,
        "credit_score": 750,
        "annual_income": 145000,
        "monthly_debts": 2100,
        "open_since": "2018-03-15",
        "last_transaction": "2024-02-10"
    },
    "CUST002": {
        "name": "Michael Chen",
        "account_type": "Standard Savings",
        "balance": 12500.00,
        "credit_score": 680,
        "annual_income": 75000,
        "monthly_debts": 1500,
        "open_since": "2020-07-22",
        "last_transaction": "2024-02-09"
    },
    "CUST003": {
        "name": "Emily Rodriguez",
        "account_type": "Business Checking",
        "balance": 89450.25,
        "credit_score": 720,
        "annual_income": 220000,
        "monthly_debts": 3500,
        "open_since": "2019-11-08",
        "last_transaction": "2024-02-11"
    }
}

MOCK_STOCK_DATA = {
    "AAPL": {"price": 182.45, "change": 2.3, "volume": 50234567},
    "GOOGL": {"price": 139.82, "change": -0.8, "volume": 25678432},
    "MSFT": {"price": 415.67, "change": 5.2, "volume": 28934561},
    "JPM": {"price": 168.23, "change": 1.5, "volume": 12345678},
    "BAC": {"price": 32.45, "change": 0.3, "volume": 45678901}
}

INTEREST_RATES = {
    "mortgage_30yr": 7.25,
    "mortgage_15yr": 6.50,
    "personal_loan": 10.99,
    "auto_loan": 6.75,
    "savings_account": 4.50,
    "cd_1yr": 5.25
}

print("✓ Mock financial database created")
print(f"  - {len(MOCK_CUSTOMER_DATABASE)} customers")
print(f"  - {len(MOCK_STOCK_DATA)} stocks")
print(f"  - {len(INTEREST_RATES)} interest rates")

---
## PART 1: BASIC REACT AGENT (20 min)

**Objective:** Build a simple ReAct agent and understand the thought-action-observation loop

### Theory: ReAct Pattern

**ReAct = Reasoning + Acting**

```
Thought: [agent's reasoning]
Action: [tool to use]
Action Input: [input for tool]
Observation: [tool output]
... (repeat)
Thought: I now know the final answer
Final Answer: [response to user]
```

### Challenge 1.1: Simple Calculator Agent

In [ ]:
# Simple Calculator Agent

import re
import operator

def calculator(expression: str) -> str:
    """
    Simple calculator that evaluates math expressions safely
    
    Supports: +, -, *, /, **, parentheses
    """
    try:
        # Remove whitespace
        expression = expression.replace(" ", "")
        
        # Safety check - only allow numbers and math operators
        if not re.match(r'^[\d\+\-\*/\(\)\.\*\s]+$', expression):
            return "Error: Invalid characters in expression. Use only numbers and +, -, *, /, **, ()"
        
        # Evaluate
        result = eval(expression)
        
        return str(result)
        
    except ZeroDivisionError:
        return "Error: Division by zero"
    except Exception as e:
        return f"Error: Could not evaluate expression - {str(e)}"


# Create calculator tool
calculator_tool = Tool(
    name="calculator",
    func=calculator,
    description="""Useful for performing mathematical calculations.
    Input should be a valid math expression using numbers and operators (+, -, *, /, **).
    Examples: "25 * 4", "100 / 3", "2 ** 8"
    Returns the numerical result."""
)

# Test calculator
print("Testing Calculator Tool:")
print(f"25 * 4 = {calculator('25 * 4')}")
print(f"100 / 3 = {calculator('100 / 3')}")
print(f"2 ** 8 = {calculator('2 ** 8')}")
print()

In [ ]:
# Create ReAct prompt template

react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

react_prompt = PromptTemplate(
    template=react_template,
    input_variables=["input", "agent_scratchpad"],
    partial_variables={
        "tools": "calculator: " + calculator_tool.description,
        "tool_names": "calculator"
    }
)

print("✓ ReAct prompt template created")

In [ ]:
# Create and test the agent

from langchain_classic.agents import create_react_agent, AgentExecutor

# Create agent
calculator_agent = create_react_agent(
    llm=llm_gpt4,
    tools=[calculator_tool],
    prompt=react_prompt
)

# Create executor
calculator_executor = AgentExecutor(
    agent=calculator_agent,
    tools=[calculator_tool],
    verbose=True,
    max_iterations=5,
    handle_parsing_errors=True
)

print("="*80)
print("CALCULATOR AGENT TEST")
print("="*80 + "\n")

# Test queries
test_queries = [
    "What is 15% of 840?",
    "If I invest $10,000 at 7.5% annual interest, how much will I have after 1 year?",
    "Calculate the monthly payment if the total is $2,400 spread over 12 months"
]

for query in test_queries:
    print(f"\nQuery: {query}")
    print("-"*80)
    
    result = calculator_executor.invoke({"input": query})
    
    print(f"\nFinal Answer: {result['output']}")
    print("="*80)

### Challenge 1.2: Understanding Agent Behavior

In [ ]:
# Agent Behavior Analysis

print("\n" + "="*80)
print("AGENT BEHAVIOR ANALYSIS")
print("="*80 + "\n")

# Query that requires multiple steps
complex_query = "What is the average of 234, 567, and 891?"

print(f"Complex Query: {complex_query}\n")
print("Observe how the agent:")
print("1. Recognizes it needs to add three numbers")
print("2. Calculates the sum")
print("3. Divides by 3 to get average")
print("4. May do this in one step or multiple steps\n")

result = calculator_executor.invoke({"input": complex_query})

print(f"\n{'='*80}")
print("ANALYSIS:")
print(f"{'='*80}")
print("Notice:")
print("- Agent broke down the problem")
print("- Used calculator tool appropriately")
print("- Arrived at correct answer")
print(f"Final Answer: {result['output']}")

### Key Takeaways - Part 1

✓ **ReAct loop**: Thought → Action → Observation → repeat  
✓ **Agent decides** when to use tools and when to stop  
✓ **Verbose mode** shows agent's reasoning process  
✓ **Error handling** in tools prevents agent crashes

---
## PART 2: CUSTOM TOOLS (25 min)

**Objective:** Build domain-specific tools for financial applications

### Challenge 2.1: Web Search Tool

In [ ]:
# Web Search Tool Integration

from langchain_community.tools import DuckDuckGoSearchRun

# Create web search tool
search = DuckDuckGoSearchRun()

web_search_tool = Tool(
    name="web_search",
    func=search.run,
    description="""Useful for finding current information on the internet.
    Input should be a search query as a plain string.
    Returns: Top search results with snippets.
    Use this when you need current information, news, or facts not in your knowledge.
    Example: "current interest rates USA" """
)

# Test search tool
print("Testing Web Search Tool:")
print("-"*80)
test_result = web_search_tool.func("current S&P 500 price")
print(f"Search result preview: {test_result[:200]}...")
print("\n✓ Web search tool created and tested")

### Challenge 2.2: Customer Lookup Tool

In [ ]:
# Customer Database Lookup Tool

def get_customer_info(customer_id: str) -> str:
    """
    Retrieve customer information from database
    
    Args:
        customer_id: Customer ID (e.g., 'CUST001')
    
    Returns:
        JSON string with customer data or error message
    """
    customer_id = customer_id.strip().upper()
    
    if customer_id not in MOCK_CUSTOMER_DATABASE:
        return f"Error: Customer ID '{customer_id}' not found. Valid IDs: {', '.join(MOCK_CUSTOMER_DATABASE.keys())}"
    
    customer = MOCK_CUSTOMER_DATABASE[customer_id]
    
    return json.dumps(customer, indent=2)


customer_lookup_tool = Tool(
    name="customer_lookup",
    func=get_customer_info,
    description="""Retrieves customer account information from the database.
    Input: Customer ID (format: CUST001, CUST002, etc.)
    Returns: JSON with customer name, account balance, credit score, income, debts, and account history.
    Use this when you need customer account details or financial profile.
    Example: "CUST001" """
)

# Test customer lookup
print("\nTesting Customer Lookup Tool:")
print("-"*80)
test_customer = customer_lookup_tool.func("CUST001")
print(test_customer)
print("\n✓ Customer lookup tool created and tested")

### Challenge 2.3: Financial Calculator Tool

In [ ]:
# Financial Calculations Tool

def financial_calculator(operation: str) -> str:
    """
    Perform common financial calculations
    
    Supported operations (as JSON):
    - {"op": "dti", "monthly_debt": X, "monthly_income": Y}
    - {"op": "ltv", "loan_amount": X, "property_value": Y}
    - {"op": "mortgage_payment", "principal": X, "rate": Y, "years": Z}
    
    Returns:
        Calculation result with explanation
    """
    try:
        data = json.loads(operation)
        op = data.get("op", "").lower()
        
        if op == "dti":
            # Debt-to-Income ratio
            monthly_debt = float(data["monthly_debt"])
            monthly_income = float(data["monthly_income"])
            dti = (monthly_debt / monthly_income) * 100
            
            assessment = "Good" if dti < 28 else "Fair" if dti < 36 else "Poor"
            
            return json.dumps({
                "operation": "Debt-to-Income Ratio",
                "dti_ratio": round(dti, 2),
                "assessment": assessment,
                "explanation": f"DTI of {dti:.1f}% is considered {assessment}. <28% is Good, 28-36% is Fair, >36% is Poor."
            }, indent=2)
        
        elif op == "ltv":
            # Loan-to-Value ratio
            loan_amount = float(data["loan_amount"])
            property_value = float(data["property_value"])
            ltv = (loan_amount / property_value) * 100
            
            assessment = "Good" if ltv < 80 else "Fair" if ltv < 90 else "Poor"
            
            return json.dumps({
                "operation": "Loan-to-Value Ratio",
                "ltv_ratio": round(ltv, 2),
                "assessment": assessment,
                "explanation": f"LTV of {ltv:.1f}% is considered {assessment}. <80% is Good, 80-90% is Fair, >90% is Poor."
            }, indent=2)
        
        elif op == "mortgage_payment":
            # Monthly mortgage payment
            principal = float(data["principal"])
            annual_rate = float(data["rate"]) / 100
            years = int(data["years"])
            
            monthly_rate = annual_rate / 12
            num_payments = years * 12
            
            if monthly_rate == 0:
                monthly_payment = principal / num_payments
            else:
                monthly_payment = principal * (monthly_rate * (1 + monthly_rate)**num_payments) / ((1 + monthly_rate)**num_payments - 1)
            
            total_paid = monthly_payment * num_payments
            total_interest = total_paid - principal
            
            return json.dumps({
                "operation": "Mortgage Payment Calculation",
                "monthly_payment": round(monthly_payment, 2),
                "total_paid": round(total_paid, 2),
                "total_interest": round(total_interest, 2),
                "explanation": f"${monthly_payment:.2f}/month for {years} years at {annual_rate*100:.2f}% APR"
            }, indent=2)
        
        else:
            return f"Error: Unknown operation '{op}'. Supported: dti, ltv, mortgage_payment"
    
    except KeyError as e:
        return f"Error: Missing required field {str(e)}"
    except ValueError as e:
        return f"Error: Invalid number format - {str(e)}"
    except Exception as e:
        return f"Error: {str(e)}"


financial_calc_tool = Tool(
    name="financial_calculator",
    func=financial_calculator,
    description="""Performs financial calculations. Input must be JSON string with operation and parameters.
    
    Operations:
    1. DTI (Debt-to-Income): {"op": "dti", "monthly_debt": 2000, "monthly_income": 8000}
    2. LTV (Loan-to-Value): {"op": "ltv", "loan_amount": 400000, "property_value": 500000}
    3. Mortgage Payment: {"op": "mortgage_payment", "principal": 400000, "rate": 7.0, "years": 30}
    
    Returns: JSON with calculation results and assessment."""
)

# Test financial calculator
print("\nTesting Financial Calculator Tool:")
print("-"*80)

# Test DTI
test_dti = financial_calc_tool.func('{"op": "dti", "monthly_debt": 2100, "monthly_income": 12083}')
print("DTI Calculation:")
print(test_dti)

print("\n" + "-"*80)

# Test mortgage payment
test_mortgage = financial_calc_tool.func('{"op": "mortgage_payment", "principal": 400000, "rate": 7.25, "years": 30}')
print("\nMortgage Payment Calculation:")
print(test_mortgage)

print("\n✓ Financial calculator tool created and tested")

### Challenge 2.4: Stock Price Tool (Mock API)

In [ ]:
# Stock Price Lookup Tool (Mock Financial API)

def get_stock_price(ticker: str) -> str:
    """
    Get current stock price and info
    
    Args:
        ticker: Stock ticker symbol (e.g., 'AAPL', 'GOOGL')
    
    Returns:
        JSON with price, change, volume
    """
    ticker = ticker.strip().upper()
    
    if ticker not in MOCK_STOCK_DATA:
        return f"Error: Ticker '{ticker}' not found. Available: {', '.join(MOCK_STOCK_DATA.keys())}"
    
    stock = MOCK_STOCK_DATA[ticker]
    
    return json.dumps({
        "ticker": ticker,
        "current_price": stock["price"],
        "change": stock["change"],
        "change_percent": round((stock["change"] / stock["price"]) * 100, 2),
        "volume": stock["volume"],
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }, indent=2)


stock_price_tool = Tool(
    name="stock_price",
    func=get_stock_price,
    description="""Gets current stock price and trading information.
    Input: Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'JPM')
    Returns: JSON with current price, change, volume.
    Use when you need stock market data or pricing information.
    Example: "AAPL" """
)

# Test stock price tool
print("\nTesting Stock Price Tool:")
print("-"*80)
test_stock = stock_price_tool.func("AAPL")
print(test_stock)

print("\n✓ Stock price tool created and tested")

### Tools Summary

In [ ]:
# Summary of all custom tools created
print("\n" + "="*80)
print("CUSTOM TOOLS SUMMARY")
print("="*80)

all_tools = [
    calculator_tool,
    web_search_tool,
    customer_lookup_tool,
    financial_calc_tool,
    stock_price_tool
]

for i, tool in enumerate(all_tools, 1):
    print(f"\n{i}. {tool.name}")
    print(f"   Description: {tool.description[:100]}...")

print(f"\n✓ Total tools created: {len(all_tools)}")
print("="*80)

---
## PART 3: MULTI-TOOL AGENT (25 min)

**Objective:** Build an agent that can use multiple tools to solve complex problems

### Challenge 3.1: Financial Advisor Agent

In [ ]:
# Multi-Tool Financial Advisor Agent

# Create enhanced ReAct prompt for multiple tools
multi_tool_template = """You are a helpful financial advisor assistant with access to various tools.

You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Important:
- For financial calculations, use the financial_calculator tool with proper JSON format
- For customer information, use customer_lookup first before making recommendations
- Use calculator for simple math, financial_calculator for DTI, LTV, mortgage calculations
- Always cite data sources in your final answer

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

multi_tool_prompt = PromptTemplate(
    template=multi_tool_template,
    input_variables=["input", "agent_scratchpad", "tools", "tool_names"]
)

# Create multi-tool agent
financial_advisor_agent = create_react_agent(
    llm=llm_gpt4,
    tools=all_tools,
    prompt=multi_tool_prompt
)

# Create executor with all tools
financial_advisor_executor = AgentExecutor(
    agent=financial_advisor_agent,
    tools=all_tools,
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True
)

print("✓ Financial Advisor Agent created with all 5 tools")

In [ ]:
# Test multi-tool agent with complex queries

print("\n" + "="*80)
print("MULTI-TOOL FINANCIAL ADVISOR AGENT")
print("="*80 + "\n")

complex_queries = [
    "What is customer CUST001's debt-to-income ratio?",
    
    "Can customer CUST002 afford a $300,000 mortgage at current 30-year rates? Calculate their monthly payment.",
    
    "What's the current price of Apple stock and how much would I need to invest to buy 100 shares?"
]

for i, query in enumerate(complex_queries, 1):
    print(f"\n{'='*80}")
    print(f"QUERY {i}: {query}")
    print(f"{'='*80}\n")
    
    result = financial_advisor_executor.invoke({"input": query})
    
    print(f"\n{'='*80}")
    print(f"FINAL ANSWER:")
    print(f"{'='*80}")
    print(result['output'])
    print(f"\n{'='*80}\n")
    
    # Pause between queries
    time.sleep(2)

### Key Takeaways - Part 3

✓ **Multi-tool agents** can combine capabilities intelligently  
✓ **Tool selection** is autonomous - agent decides which tool when  
✓ **Data flows** between tools (customer lookup → calculator)  
✓ **Complex problems** broken down into steps automatically

---
## PART 4: CONVERSATIONAL AGENT WITH MEMORY (20 min)

**Objective:** Add memory so agent can reference previous conversation turns

### Challenge 4.1: Agent with Conversation Memory

In [ ]:
# Conversational Agent with Memory

from langchain_classic.memory import ConversationBufferMemory

# Create memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Create conversational ReAct prompt
conversational_template = """You are a helpful financial advisor assistant having a conversation with a customer.

Previous conversation:
{chat_history}

You have access to the following tools:

{tools}

Use the following format:

Question: the current question from the customer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the customer

Important:
- Reference previous conversation when relevant
- Remember customer details mentioned earlier
- Provide personalized responses based on conversation history

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

conversational_prompt = PromptTemplate(
    template=conversational_template,
    input_variables=["input", "chat_history", "agent_scratchpad", "tools", "tool_names"]
)

# Create conversational agent
conversational_agent = create_react_agent(
    llm=llm_gpt4,
    tools=all_tools,
    prompt=conversational_prompt
)

# Create executor with memory
conversational_executor = AgentExecutor(
    agent=conversational_agent,
    tools=all_tools,
    memory=memory,
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True
)

print("✓ Conversational agent with memory created")

In [ ]:
# Test conversational agent

print("\n" + "="*80)
print("CONVERSATIONAL AGENT WITH MEMORY")
print("="*80 + "\n")

# Simulated multi-turn conversation
conversation = [
    "Hi, I'm interested in getting a mortgage. My customer ID is CUST001.",
    "What's my current debt-to-income ratio?",
    "If I take out a $500,000 mortgage at 7.25% for 30 years, what would my new DTI be?",
    "Is that considered good or should I pay down some debt first?"
]

for i, message in enumerate(conversation, 1):
    print(f"\n{'='*80}")
    print(f"USER (Turn {i}): {message}")
    print(f"{'='*80}\n")
    
    result = conversational_executor.invoke({"input": message})
    
    print(f"\n{'='*80}")
    print(f"ASSISTANT:")
    print(f"{'='*80}")
    print(result['output'])
    print(f"\n{'='*80}\n")
    
    time.sleep(2)

# Show conversation history
print("\n" + "="*80)
print("CONVERSATION MEMORY")
print("="*80)
print(memory.load_memory_variables({}))

### Key Takeaways - Part 4

✓ **Memory enables** multi-turn conversations  
✓ **Agent remembers** customer ID, previous calculations  
✓ **Context-aware responses** reference earlier exchanges  
✓ **ConversationBufferMemory** stores full history

---
## PART 5: AGENT PROMPT OPTIMIZATION (15 min)

**Objective:** Improve agent reliability and reduce errors

### Challenge 5.1: Optimizing Tool Descriptions

In [ ]:
# Improved Tool Descriptions

# BEFORE: Vague description
calculator_tool_bad = Tool(
    name="calc",
    func=calculator,
    description="does math"
)

# AFTER: Clear, detailed description
calculator_tool_good = Tool(
    name="calculator",
    func=calculator,
    description="""Performs basic mathematical calculations.
    
    When to use:
    - Simple arithmetic (addition, subtraction, multiplication, division)
    - Percentages (e.g., "15% of 840" → "0.15 * 840")
    - Powers (e.g., "2 to the power of 8" → "2 ** 8")
    
    Input format: Math expression as string (e.g., "25 * 4", "100 / 3")
    Output: Numerical result
    
    Examples:
    - "50 + 30" → "80"
    - "0.15 * 1000" → "150.0"
    - "2 ** 10" → "1024"
    
    Do NOT use for:
    - Financial ratios (use financial_calculator instead)
    - Complex financial calculations (use financial_calculator instead)
    """
)

print("="*80)
print("TOOL DESCRIPTION OPTIMIZATION")
print("="*80)
print("\nBAD Description:")
print(f"  {calculator_tool_bad.description}")
print("\nGOOD Description:")
print(f"  {calculator_tool_good.description}")
print("\nKey Improvements:")
print("  1. ✓ Explains WHEN to use the tool")
print("  2. ✓ Shows INPUT format clearly")
print("  3. ✓ Provides multiple EXAMPLES")
print("  4. ✓ States what NOT to use it for")
print("="*80)

### Challenge 5.2: Better System Prompts

In [ ]:
# Optimized Agent System Prompt

# BEFORE: Generic prompt
generic_prompt = """Answer questions using tools.

Tools: {tools}

Question: {input}"""

# AFTER: Specific, structured prompt with guidelines
optimized_prompt_template = """You are an expert financial advisor AI assistant helping customers with banking and financial questions.

AVAILABLE TOOLS:
{tools}

TOOL SELECTION GUIDELINES:
1. customer_lookup: Always use FIRST when customer ID is mentioned
2. financial_calculator: Use for DTI, LTV, mortgage payments (requires JSON input)
3. calculator: Use for simple math only
4. stock_price: Use for stock market data
5. web_search: Use only when you need current information not available in other tools

RESPONSE GUIDELINES:
- Always verify customer identity before sharing account details
- Cite specific numbers and sources
- Explain financial terms clearly
- If uncertain, say so rather than guessing
- Format monetary values with $ and 2 decimal places

FORMAT:
Question: the input question
Thought: what you should do next
Action: tool to use from [{tool_names}]
Action Input: input for the tool
Observation: tool result
... (repeat as needed)
Thought: I have enough information to answer
Final Answer: your response to the customer

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

print("\n" + "="*80)
print("SYSTEM PROMPT OPTIMIZATION")
print("="*80)
print("\nKey Improvements:")
print("  1. ✓ Defines agent's role and expertise")
print("  2. ✓ Provides tool selection guidelines")
print("  3. ✓ Sets response quality standards")
print("  4. ✓ Includes formatting requirements")
print("  5. ✓ Reduces hallucinations with specific instructions")
print("="*80)

### Key Takeaways - Part 5

✓ **Tool descriptions** critically impact agent behavior  
✓ **Clear examples** reduce tool selection errors  
✓ **System prompts** guide overall agent quality  
✓ **Specific instructions** reduce hallucinations

---
## PART 6: ADVANCED AGENT PATTERNS (15 min)

**Objective:** Implement plan-and-execute and self-correction patterns

### Challenge 6.1: Plan-and-Execute Agent

In [ ]:
# Plan-and-Execute Pattern

def plan_and_execute(query: str, tools: List[Tool], llm) -> Dict[str, Any]:
    """
    Plan-and-Execute pattern: First create a plan, then execute each step
    
    Benefits:
    - More predictable behavior
    - Better for complex multi-step tasks
    - Can show plan to user for approval
    """
    
    # Step 1: Create a plan
    planning_prompt = f"""Given this query, create a step-by-step plan to answer it.

Query: {query}

Available tools: {', '.join([t.name for t in tools])}

Create a numbered plan with specific steps. Each step should:
- Be actionable
- Specify which tool to use
- Include what information to gather

Plan:"""
    
    plan_response = llm.invoke(planning_prompt)
    plan_text = plan_response.content if hasattr(plan_response, 'content') else str(plan_response)
    
    print("="*80)
    print("PLAN:")
    print("="*80)
    print(plan_text)
    print("="*80 + "\n")
    
    # Step 2: Execute the plan (simulated - in production, parse and execute each step)
    execution_prompt = f"""Following this plan, answer the query using the tools.

Plan:
{plan_text}

Original Query: {query}

Execute the plan step by step using the ReAct format."""
    
    # Create temporary agent for execution
    exec_agent = create_react_agent(llm, tools, multi_tool_prompt)
    exec_executor = AgentExecutor(
        agent=exec_agent,
        tools=tools,
        verbose=True,
        max_iterations=15
    )
    
    result = exec_executor.invoke({"input": execution_prompt})
    
    return {
        "query": query,
        "plan": plan_text,
        "result": result["output"]
    }


# Test plan-and-execute
print("\n" + "="*80)
print("PLAN-AND-EXECUTE PATTERN")
print("="*80 + "\n")

complex_query = """Compare customer CUST001 and CUST002's financial profiles. 
For each, calculate their DTI ratio and determine if they could afford a $400,000 mortgage."""

result = plan_and_execute(complex_query, all_tools, llm_gpt4)

print("\n" + "="*80)
print("FINAL RESULT:")
print("="*80)
print(result["result"])
print("="*80)

### Challenge 6.2: Self-Correction Pattern

In [ ]:
# Self-Correction Pattern

def agent_with_self_correction(query: str, executor: AgentExecutor) -> Dict[str, Any]:
    """
    Self-correction pattern: Agent reviews its own answer and corrects if needed
    """
    
    # Step 1: Get initial answer
    print("="*80)
    print("STEP 1: Initial Answer")
    print("="*80 + "\n")
    
    initial_result = executor.invoke({"input": query})
    initial_answer = initial_result["output"]
    
    print(f"\nInitial Answer: {initial_answer}\n")
    
    # Step 2: Self-critique
    print("="*80)
    print("STEP 2: Self-Critique")
    print("="*80 + "\n")
    
    critique_prompt = f"""Review this answer for accuracy and completeness:

Question: {query}
Answer: {initial_answer}

Identify any:
1. Factual errors
2. Missing information
3. Unclear explanations
4. Calculation mistakes

Critique:"""
    
    critique = llm_gpt4.invoke(critique_prompt)
    critique_text = critique.content if hasattr(critique, 'content') else str(critique)
    print(f"Critique: {critique_text}\n")
    
    # Step 3: Revise if needed
    if "error" in critique_text.lower() or "missing" in critique_text.lower() or "incorrect" in critique_text.lower():
        print("="*80)
        print("STEP 3: Revised Answer")
        print("="*80 + "\n")
        
        revision_prompt = f"""{query}

Previous answer: {initial_answer}
Issues found: {critique_text}

Provide a corrected and improved answer."""
        
        revised_result = executor.invoke({"input": revision_prompt})
        final_answer = revised_result["output"]
    else:
        final_answer = initial_answer
        print("No revision needed - initial answer is good!\n")
    
    return {
        "query": query,
        "initial_answer": initial_answer,
        "critique": critique_text,
        "final_answer": final_answer
    }


# Test self-correction
print("\n" + "="*80)
print("SELF-CORRECTION PATTERN")
print("="*80 + "\n")

test_query = "What is customer CUST001's monthly income and can they afford a $3,000/month payment?"

result = agent_with_self_correction(test_query, financial_advisor_executor)

print("\n" + "="*80)
print("SELF-CORRECTION SUMMARY:")
print("="*80)
print(f"Initial: {result['initial_answer'][:200]}...")
print(f"\nCritique: {result['critique'][:200]}...")
print(f"\nFinal: {result['final_answer'][:200]}...")
print("="*80)